# 🔍 X(Twitter) 凍結アカウントチェッカー

Googleスプレッドシートのアカウント一覧をチェックして、凍結・存在しないアカウントを自動で色付けします。

---

## 📋 使い方

1. **設定セル**でスプレッドシートURLなどを入力
2. **「ランタイム」→「すべてのセルを実行」**
3. Googleアカウント認証で「許可」
4. 完了まで待つ（100件で約5分）

### 色の意味
- 🔴 薄い赤: 凍結
- ⚪ グレー: 存在しない
- 🟡 薄い黄色: 判定不能

In [ ]:
#@title ⚙️ 設定（ここを編集してください）
#@markdown ---
#@markdown ### スプレッドシートのURL
#@markdown ブラウザからコピペしてください
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/xxxxx/edit" #@param {type:"string"}

#@markdown ---
#@markdown ### アカウントIDが入っている列
ACCOUNT_COLUMN = "A" #@param ["A", "B", "C", "D", "E", "F", "G", "H"]

#@markdown ---
#@markdown ### 1行目をスキップする（ヘッダー行がある場合）
SKIP_HEADER = True #@param {type:"boolean"}

print("✅ 設定完了！")
print(f"   URL: {SPREADSHEET_URL[:50]}...")
print(f"   列: {ACCOUNT_COLUMN}")
print(f"   ヘッダースキップ: {SKIP_HEADER}")

In [ ]:
#@title 📦 ライブラリインストール（初回のみ時間がかかります）
print("📦 インストール中...")
!pip install -q gspread google-auth nest_asyncio playwright
!playwright install chromium --with-deps 2>/dev/null
print("✅ インストール完了！")

In [ ]:
#@title 🔐 Googleアカウント認証
import nest_asyncio
nest_asyncio.apply()

from google.colab import auth
import gspread
from google.auth import default

print("🔐 Googleアカウントに接続中...")
print("   ポップアップが出たら「許可」をクリック")
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("✅ 認証完了！")

In [ ]:
#@title 📊 スプレッドシート読み込み
import re

def extract_id(v):
    if not v: return None
    v = v.strip()
    m = re.search(r'(?:x\.com|twitter\.com)/([a-zA-Z0-9_]+)', v)
    if m: return m.group(1)
    if v.startswith('@'): return v[1:].strip()
    if re.match(r'^[a-zA-Z0-9_]+$', v): return v
    return None

match = re.search(r'/d/([a-zA-Z0-9-_]+)', SPREADSHEET_URL)
if not match:
    raise ValueError("❌ URLが正しくありません")

spreadsheet = gc.open_by_key(match.group(1))
worksheet = spreadsheet.worksheets()[0]  # 最初のシート
print(f"📊 シート名: {worksheet.title}")

col_num = ord(ACCOUNT_COLUMN.upper()) - ord('A') + 1
all_values = worksheet.col_values(col_num)
start_row = 2 if SKIP_HEADER else 1

accounts = [(i+start_row, extract_id(v)) for i,v in enumerate(all_values[start_row-1:]) if extract_id(v)]
print(f"✅ {len(accounts)} 件のアカウントを読み込みました")
if accounts:
    print(f"   例: {[a[1] for a in accounts[:3]]}")

In [ ]:
#@title 🚀 凍結チェック実行
from playwright.async_api import async_playwright
from datetime import datetime

async def run_check():
    results = []
    frozen = 0
    not_exist = 0
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        
        for i, (row, acc) in enumerate(accounts):
            page = await browser.new_page()
            try:
                await page.goto(f"https://x.com/{acc}", timeout=30000)
                await page.wait_for_timeout(3000)
                c = await page.content()
                
                if "Account suspended" in c:
                    print(f"❄️ {acc}: 凍結")
                    results.append((row, acc, "frozen"))
                    frozen += 1
                elif "This account doesn" in c:
                    print(f"👻 {acc}: 存在しない")
                    results.append((row, acc, "not_exist"))
                    not_exist += 1
                elif "Following" in c and "Followers" in c:
                    results.append((row, acc, "normal"))
                else:
                    print(f"❓ {acc}: 判定不能")
                    results.append((row, acc, "unknown"))
            except:
                results.append((row, acc, "error"))
            finally:
                await page.close()
            
            if (i + 1) % 10 == 0:
                print(f"⏳ 進捗: {i+1}/{len(accounts)}")
        
        await browser.close()
    
    return results, frozen, not_exist

print(f"🚀 チェック開始: {len(accounts)} 件")
print(f"⏱️ 開始時刻: {datetime.now().strftime('%H:%M:%S')}")
print("=" * 40)

results, frozen_count, not_exist_count = await run_check()

print("=" * 40)
print(f"✅ チェック完了！ {datetime.now().strftime('%H:%M:%S')}")
print(f"❄️ 凍結: {frozen_count} 件")
print(f"👻 存在しない: {not_exist_count} 件")

In [ ]:
#@title 📝 結果をスプレッドシートに書き込み
from datetime import datetime

print("📝 スプレッドシートに書き込み中...")

COLOR_FROZEN = {'red': 1, 'green': 0.8, 'blue': 0.8}
COLOR_NOT_EXIST = {'red': 0.85, 'green': 0.85, 'blue': 0.85}
COLOR_UNKNOWN = {'red': 1, 'green': 1, 'blue': 0.8}

col_idx = ord(ACCOUNT_COLUMN.upper()) - ord('A')
reqs = []

for row, acc, st in results:
    if st == "frozen":
        color = COLOR_FROZEN
    elif st == "not_exist":
        color = COLOR_NOT_EXIST
    elif st == "unknown":
        color = COLOR_UNKNOWN
    else:
        continue
    
    reqs.append({
        'repeatCell': {
            'range': {
                'sheetId': worksheet.id,
                'startRowIndex': row - 1,
                'endRowIndex': row,
                'startColumnIndex': col_idx,
                'endColumnIndex': col_idx + 1
            },
            'cell': {
                'userEnteredFormat': {'backgroundColor': color},
                'note': f"{st} ({datetime.now().strftime('%Y/%m/%d %H:%M')})"
            },
            'fields': 'userEnteredFormat.backgroundColor,note'
        }
    })

if reqs:
    spreadsheet.batch_update({'requests': reqs})
    print(f"✅ {len(reqs)} 件のセルを更新しました")
else:
    print("ℹ️ 更新が必要なセルはありませんでした")

print("")
print("🎉 完了！スプレッドシートを確認してください")